In [63]:
from datetime import datetime
import numpy as np
import numpy as numpy
import pandas as pd
import pylab
import calendar
from scipy import stats
import seaborn as sns
from sklearn import model_selection, preprocessing
from scipy.stats import kendalltau
import warnings
import matplotlib.pyplot as plt
import pandas

In [64]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D,MaxPooling1D
from keras.layers import Dropout, BatchNormalization, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
K.set_image_dim_ordering('tf')

In [65]:
#################
##  READ DATA  ##
#################

# Load train, Prop and sample
print('Loading train, prop and sample data')
train = pd.read_csv("../Inputs/train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv('../Inputs/properties_2016.csv')
sample = pd.read_csv('../Inputs/sample_submission.csv')
 
print('Fitting Label Encoder on properties')
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))
        
#Create df_train and x_train y_train from that
print('Creating training set:')
df_train = train.merge(prop, how='left', on='parcelid')

###########################################################

Loading train, prop and sample data


c:\users\vishwas\anaconda2\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting Label Encoder on properties
Creating training set:


In [66]:
###########################################################
df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train["transactiondate_year"] = df_train["transactiondate"].dt.year
df_train["transactiondate_month"] = df_train["transactiondate"].dt.month
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
df_train["transactiondate"] = df_train["transactiondate"].dt.day

In [67]:
###########################################

print('Fill  NA/NaN values using suitable method' )
#df_train.fillna(df_train.mean(),inplace = True)
df_train.fillna(-1.0)

#df_train =df_train[ df_train.logerror > -0.4005 ]
#df_train=df_train[ df_train.logerror < 0.412 ]

Fill  NA/NaN values using suitable method


,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate_year,transactiondate_month,transactiondate_quarter
0,11016594,0.0276,1,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,...,360170.0,2015.0,237416.0,6735.88,0,-1.0,6.037107e+13,2016,1,1
1,14366692,-0.1684,1,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,...,585529.0,2015.0,239071.0,10153.02,0,-1.0,-1.000000e+00,2016,1,1
2,12098116,-0.0040,1,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,...,119906.0,2015.0,57912.0,11484.48,0,-1.0,6.037464e+13,2016,1,1
3,12643413,0.0218,2,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,244880.0,2015.0,73362.0,3048.74,0,-1.0,6.037296e+13,2016,1,1
4,14432541,-0.0050,2,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,...,434551.0,2015.0,264977.0,5488.96,0,-1.0,6.059042e+13,2016,1,1
5,11509835,-0.2705,2,1.0,-1.0,-1.0,4.0,4.0,-1.0,1.0,...,2447951.0,2015.0,1567301.0,27126.57,0,-1.0,6.037621e+13,2016,1,1
6,12286022,0.0440,2,-1.0,-1.0,-1.0,1.0,2.0,-1.0,7.0,...,111521.0,2015.0,46972.0,2304.97,0,-1.0,6.037542e+13,2016,1,1
7,17177301,0.1638,2,-1.0,-1.0,-1.0,2.5,3.0,-1.0,-1.0,...,306000.0,2015.0,199000.0,3745.50,0,-1.0,6.111003e+13,2016,1,1
8,14739064,-0.0030,2,-1.0,-1.0,-1.0,1.0,2.0,-1.0,-1.0,...,210064.0,2015.0,143230.0,2172.88,0,-1.0,6.059042e+13,2016,1,1
9,14677559,0.0843,3,-1.0,-1.0,-1.0,2.0,2.0,-1.0,-1.0,...,190960.0,2015.0,80983.0,1940.26,0,-1.0,6.059063e+13,2016,1,1


In [68]:
print('Create x_train and y_train from df_train' )
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode','fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train["logerror"]

Create x_train and y_train from df_train


In [69]:
y_mean = np.mean(y_train)
print(x_train.shape, y_train.shape)
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
# Create df_test and test set
print('Creating df_test  :')
sample['parcelid'] = sample['ParcelId']

print("Merge Sample with property data :")
df_test = sample.merge(prop, on='parcelid', how='left')


(90275, 56) (90275,)
Creating df_test  :
Merge Sample with property data :


In [70]:

########################
df_test["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
df_test["transactiondate"] = df_test["transactiondate"].dt.day     


In [71]:

#################################


x_test = df_test[train_columns]

print('Shape of x_test:', x_test.shape)
print("Preparing x_test:")
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
  
#print("Bind x_test to float32:")
#x_test = x_test.values.astype(np.float32, copy=False)




Shape of x_test: (2985217, 56)
Preparing x_test:


In [72]:
###################
##  PREPROCESS  ##
###################


#############################Imputer##################

from sklearn.preprocessing import Imputer
imputer= Imputer()
imputer.fit(x_train.iloc[:, :])
x_train = imputer.transform(x_train.iloc[:, :])
imputer.fit(x_test.iloc[:, :])
x_test = imputer.transform(x_test.iloc[:, :])

#########################Standard Scalar##############

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

################################################




In [73]:
print("x_train shape:",x_train.shape)

x_train shape: (90275, 56)


In [74]:
###################
##  RUN NETWORK  ##
###################

count = 8
len_x=int(x_train.shape[1])
print("len_x is:",len_x)

## The model- Uses a few dense layers on top of LSTM and CNN layers. 

model = Sequential()
model.add(Conv1D(filters=20, kernel_size=10 ,strides=10,     
                  input_shape = (len_x,1), kernel_initializer= 'uniform',      
                  activation= 'relu'))
model.add(MaxPooling1D(pool_size=2, strides=10, padding='same'))
model.add(Dropout(.17))
model.add(LSTM(units = 150 , kernel_initializer = 'normal', activation = 'softmax',return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(units = 150 , kernel_initializer = 'normal', activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(.32))
model.add(Dense(units = 75, kernel_initializer = 'normal', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(.22))
model.add(Dense(units = 25, kernel_initializer = 'normal', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(.22))
model.add(Flatten())
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_absolute_error', optimizer='rmsprop')

model.summary()




len_x is: 56
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 5, 20)             220       
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 1, 20)             0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 1, 20)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 150)            102600    
_________________________________________________________________
batch_normalization_43 (Batc (None, 1, 150)            600       
_________________________________________________________________
dropout_57 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
dense_44 (Dense)             (None, 1, 150)            22650   

In [75]:
print(len(x_train), len(y_train))
x = np.array(x_train)
x = np.expand_dims(x, axis=2)
y = np.array(y_train)
#y = y.reshape(y_train.shape[0],)
print(x.shape,y.shape)

90275 90275
(90275, 56, 1) (90275,)


In [76]:


model.fit(x,y, batch_size = 128, epochs = 20, verbose=1)

print("x_test.shape:",x_test.shape)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)


#######################################################################################

print( "\nPreparing results for write :" )

Epoch 1/1
90275/90275 [==============================] - 49s 538us/step - loss: 0.0728
x_test.shape: (2985217, 56)

Preparing results for write :


In [77]:
y_pred_ann = model.predict(x_test)
print(y_pred_ann.shape)
y_pred = y_pred_ann.flatten()
print(y_pred.shape)

(2985217, 1)
(2985217,)


In [78]:
#####################
##  WRITE RESULTS  ##
#####################

#y_pred = y_pred_ann.flatten()

#output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
output = pd.DataFrame({'ParcelId': prop['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]

print( "\nWriting results to disk:" )
output.to_csv('Output_{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print( "\nFinished!" )


Writing results to disk:

Finished!
